In [59]:
import pandas as pd
import numpy as np

In [60]:
X_train = pd.read_parquet("C:\\Users\\Cash Crusaders\\Desktop\\My Portfolio\\Projects\\Data Science Projects\\Data Science Project 11 - Laptop Price Prediction\\dataset\\feature_engineered_data\\X_train_df.parquet.gzip")
X_test = pd.read_parquet("C:\\Users\\Cash Crusaders\\Desktop\\My Portfolio\\Projects\\Data Science Projects\\Data Science Project 11 - Laptop Price Prediction\\dataset\\feature_engineered_data\\X_test_df.parquet")
y_train = pd.read_csv("C:\\Users\\Cash Crusaders\\Desktop\\My Portfolio\\Projects\\Data Science Projects\\Data Science Project 11 - Laptop Price Prediction\\dataset\\feature_engineered_data\\y_train_df.csv")
y_test = pd.read_csv("C:\\Users\\Cash Crusaders\\Desktop\\My Portfolio\\Projects\\Data Science Projects\\Data Science Project 11 - Laptop Price Prediction\\dataset\\feature_engineered_data\\y_test_df.csv")


In [61]:
X_train.head(2)
# drop
# Cpu
# Gpu


,Unnamed: 0,Company,TypeName,Cpu,Ram,Gpu,Weight,Touchscreen,Ips,ppi,Cpu Name,Cpu brand,HDD,SSD,Gpu brand,os
1259,1259,MSI,Gaming,Intel Core i7 6700HQ 2.6GHz,8,Nvidia GeForce GTX 960M,2.40,0,0,141.211998,Intel Core i7,Intel Core i7,1000,128,Nvidia,Windows
1007,1007,HP,Ultrabook,Intel Core i7 6600U 2.6GHz,8,Intel HD Graphics 520,1.43,0,0,157.350512,Intel Core i7,Intel Core i7,0,256,Intel,Windows


In [62]:
y_train.head(2)

,Unnamed: 0,series
0,1259,11.089517
1,1007,11.254190


In [63]:
X_train.drop(columns=['Unnamed: 0'],inplace=True)
X_train.drop(columns=['Cpu'],inplace=True)
X_train.drop(columns=['Gpu'],inplace=True)
X_train.drop(columns=['Cpu Name'],inplace=True)
y_train.drop(columns=['Unnamed: 0'],inplace=True)

In [64]:

X_train.head(2)

,Company,TypeName,Ram,Weight,Touchscreen,Ips,ppi,Cpu brand,HDD,SSD,Gpu brand,os
1259,MSI,Gaming,8,2.40,0,0,141.211998,Intel Core i7,1000,128,Nvidia,Windows
1007,HP,Ultrabook,8,1.43,0,0,157.350512,Intel Core i7,0,256,Intel,Windows


In [65]:
y_train.head(2)

,series
0,11.089517
1,11.254190


In [66]:
y_train.tail(2)

,series
1039,10.776844
1040,10.881314


In [67]:
X_test.head(2)

,Unnamed: 0,Company,TypeName,Cpu,Ram,Gpu,Weight,Touchscreen,Ips,ppi,Cpu Name,Cpu brand,HDD,SSD,Gpu brand,os
1233,1233,MSI,Gaming,Intel Core i7 7700HQ 2.8GHz,16,Nvidia GeForce GTX 1060,2.70,0,0,127.335675,Intel Core i7,Intel Core i7,1000,256,Nvidia,Windows
1030,1030,HP,Notebook,Intel Core i5 7200U 2.5GHz,4,Intel HD Graphics 620,1.64,0,0,111.935204,Intel Core i5,Intel Core i5,0,256,Intel,Windows


In [68]:
y_test.head(2)

,Unnamed: 0,series
0,1233,11.352069
1,1030,10.855945


In [69]:
X_test.drop(columns=['Unnamed: 0'],inplace=True)
X_test.drop(columns=['Cpu'],inplace=True)
X_test.drop(columns=['Gpu'],inplace=True)
X_test.drop(columns=['Cpu Name'],inplace=True)
y_test.drop(columns=['Unnamed: 0'],inplace=True)

In [81]:
import mlflow

#set the tracking uri - needed for the ssqlite 
mlflow.set_tracking_uri("sqlite:///demo.db")

In [82]:
#set the experiment
mlflow.set_experiment("demo-experiment-1")

<Experiment: artifact_location='./mlruns/1', creation_time=1680700375622, experiment_id='1', last_update_time=1680700375622, lifecycle_stage='active', name='demo-experiment-1', tags={}>

In [72]:
from sklearn.base import RegressorMixin
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import r2_score,mean_absolute_error

from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor,AdaBoostRegressor,ExtraTreesRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
import xgboost
from sklearn.metrics import mean_squared_error, r2_score

In [73]:
from zenml.steps import step, Output, step_output

### Model Training

In [84]:
def convert_input(X):
    TypeName_map=X['TypeName'].value_counts().to_dict()
    X['TypeName']=X['TypeName'].map(TypeName_map)
    # company
    company_map=X['Company'].value_counts().to_dict()
    X['Company']=X['Company'].map(company_map)
    #Cpu brand
    cpu_brand_map=X['Cpu brand'].value_counts().to_dict()
    X['Cpu brand']=X['Cpu brand'].map(cpu_brand_map)
    # Gpu brand
    gpu_brand_map=X['Gpu brand'].value_counts().to_dict()
    X['Gpu brand']=X['Gpu brand'].map(gpu_brand_map)
    # os
    os_map=X['os'].value_counts().to_dict()
    X['os']=X['os'].map(os_map)
    return X

In [85]:
def train_model(X_train:pd.DataFrame, y_train:pd.DataFrame, X_val:pd.DataFrame, y_val:pd.DataFrame)->Output(
    TRAINING_EXPERIMENT_NAME = str
):
    
    mlflow.sklearn.autolog()
    # X_train_transformed = convert_input(X_train)
    # X_val_transformed = convert_input(X_val)
    for model_class in (RandomForestRegressor,GradientBoostingRegressor,AdaBoostRegressor,
                        ExtraTreesRegressor,SVR,XGBRegressor,DecisionTreeRegressor,KNeighborsRegressor,
                        LinearRegression,Ridge,Lasso):
        
        with mlflow.start_run(nested=True):
            #tags of the runs
            mlflow.set_tag("ml-engineer", "thobela")
            # column transformer iss used to transform columns,we are hot encoding 5 columns(the ones that have categorical values) usingg thei indexes
            
            column_transformer = ColumnTransformer(transformers=[
                ('col_tnf',OneHotEncoder(sparse=False,drop='first'),[0,1,7,10,11])
            ],remainder='passthrough')
            X_train_tranformed = np.array(column_transformer.fit_transform(X_train))
            X_val_tranformed = np.array(column_transformer.transform(X_val))
            # Use a pipeline to enclose everything to make it easier for deployment
            mlmodel = model_class()

            # create a pipeline
            # ml_pipeline = Pipeline([
            #     ('step1',column_transformer),
            #     ('step2',mlmodel)
            # ])
            mlmodel.fit(X_train_tranformed, y_train)

            y_pred = mlmodel.predict(X_val_tranformed)
            rmse = mean_squared_error(y_val, y_pred, squared=False)
            r2 = r2_score(y_val,y_pred)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2_score", r2)
            print("R2 Score : ",r2)
            print("validation-rmse: ",rmse)
            mlflow.end_run()
    print("\n")
    print('Sucessfully trained...')
    

In [76]:
X_train.head(1)

,Company,TypeName,Ram,Weight,Touchscreen,Ips,ppi,Cpu brand,HDD,SSD,Gpu brand,os
1259,MSI,Gaming,8,2.4,0,0,141.211998,Intel Core i7,1000,128,Nvidia,Windows


In [77]:
X_test.head(1)

,Company,TypeName,Ram,Weight,Touchscreen,Ips,ppi,Cpu brand,HDD,SSD,Gpu brand,os
1233,MSI,Gaming,16,2.7,0,0,127.335675,Intel Core i7,1000,256,Nvidia,Windows


In [78]:
ct = column_transformer = ColumnTransformer(transformers=[
                ('col_tnf',OneHotEncoder(sparse=False,drop='first'),[0,1,7,10,11])
            ],remainder='passthrough')
X_train_tranformed = np.array(ct.fit_transform(X_train))
X_test_tranformed = np.array(ct.transform(X_test))

In [79]:
X_test_tranformed.shape

(261, 38)

In [80]:
X_train_tranformed.shape

(1041, 38)

In [86]:
train_model(X_train,y_train,X_test,y_test)

2023/04/07 15:56:22 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Anaconda\envs\machine-learning-env\lib\site-packages\mlflow\utils\autologging_utils\safety.py:533: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel()."


R2 Score :  0.8806550096276546
validation-rmse:  0.207282714727161


c:\Anaconda\envs\machine-learning-env\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


R2 Score :  0.8615984902409635
validation-rmse:  0.22321912674711508


c:\Anaconda\envs\machine-learning-env\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


R2 Score :  0.7999465243428561
validation-rmse:  0.26837004654143415


2023/04/07 15:56:58 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Anaconda\envs\machine-learning-env\lib\site-packages\mlflow\utils\autologging_utils\safety.py:533: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel()."


R2 Score :  0.8647216105805524
validation-rmse:  0.22068621340837452


c:\Anaconda\envs\machine-learning-env\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


R2 Score :  0.5292633017939731
validation-rmse:  0.4116707582950574
R2 Score :  0.8696061806583709
validation-rmse:  0.2166653656680409
R2 Score :  0.8000474793733572
validation-rmse:  0.26830232283630756
R2 Score :  0.7042926066025396
validation-rmse:  0.3262812237517812
R2 Score :  0.7958532530922593
validation-rmse:  0.2711016887308244
R2 Score :  0.795377785078817
validation-rmse:  0.2714172098372369
R2 Score :  0.4501505300394437
validation-rmse:  0.444920991470731


Sucessfully trained...


In [87]:
import pandas as pd

import mlflow
import numpy as np
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe, space_eval
from hyperopt.pyll import scope
from sklearn.metrics import mean_squared_error
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType
from zenml.steps import step, Output, step_output
from zenml.client import Client
import sklearn

In [155]:
#MLFLOW_TRACKING_URI = get_tracking_uri()
MLFLOW_TRACKING_URI = "sqlite:///demo.db"
# assign a name for this hyperparameter tuning experiment
TRAINING_EXPERIMENT_NAME = "demo-experiment-1"
EXPERIMENT_NAME = "hyper-optimization-experiment-1"
# set the stack experiment tracking uri to be the current working one
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
# set the current experiment name to be the current hyperparameter tuning one
mlflow.set_experiment(EXPERIMENT_NAME)

<Experiment: artifact_location='./mlruns/2', creation_time=1680706923403, experiment_id='2', last_update_time=1680706923403, lifecycle_stage='active', name='hyper-optimization-experiment-1', tags={}>

In [182]:
def fix_param_datatypes(param_dict):
    correct_param_dict={}
    for k, v in param_dict.items():
        if v != 'None' and v != 'True' and v != 'False' and v != 'squared_error' and k != 'criterion':
            correct_param_dict[k] = float(v)
        else:
            correct_param_dict[k] = str(v)
    return correct_param_dict

### Model Tuning

In [188]:

# a function for optimizing the chosen number of models from the model runs
def optimize_model(X_train:pd.DataFrame, y_train:pd.Series, X_val:pd.DataFrame, y_val:pd.Series, params, tags):
   ''' X_train, X_test,y_train, y_test : for training and testing the models with new parameters
    params: chosen paramters from a specific run
   '''
   estimator_name = tags['estimator_name']
   pool_of_models = {
      "RandomForestRegressor":RandomForestRegressor,
      "GradientBoostingRegressor":GradientBoostingRegressor,
      "AdaBoostRegressor":AdaBoostRegressor,
      "ExtraTreesRegressor":ExtraTreesRegressor,
      "SVR":SVR,
      "XGBRegressor":XGBRegressor,
      "DecisionTreeRegressor":DecisionTreeRegressor,
      "KNeighborsRegressor":KNeighborsRegressor,
      "LinearRegression":LinearRegression,
      "Ridge":Ridge,
      "Lasso":Lasso
   }
   model_class = pool_of_models[estimator_name]
   param_dict = fix_param_datatypes(params)
   # have a pool of values that will take on the different parameters, a seach space for values for the model
   # assign as value pool space for different parameters where the models will take different parameters from
        
   with mlflow.start_run():

      # start the logging of the experiment
      mlflow.sklearn.autolog()
      # column transformer iss used to transform columns,we are hot encoding 5 columns(the ones that have categorical values) usingg thei indexes
      column_transformer = ColumnTransformer(transformers=[
            ('col_tnf',OneHotEncoder(sparse=False,drop='first'),[0,1,7,10,11])
      ],remainder='passthrough')
      X_train_tranformed = np.array(column_transformer.fit_transform(X_train))
      X_val_tranformed = np.array(column_transformer.transform(X_val))
      # Use a pipeline to enclose everything to make it easier for deployment
      # use the run paramters given to compoye the best paramters from the space pool
      #params = space_eval(global_pool_space, params)
      # assign those parameters to the model API class
      mlmodel = model_class()

      # fit the model with new parameters
      mlmodel.fit(X_train_tranformed, y_train)
      # do predictions with new parameters
      y_pred = mlmodel.predict(X_val_tranformed)
      # get the rsme and log it
      rmse = mean_squared_error(y_val, y_pred, squared=False)
      r2 = r2_score(y_val,y_pred)
      mlflow.log_metric("r2_score", r2)
      mlflow.log_metric("rmse", rmse)
      # mlflow.log_artifact("artifact/preprocessor.b")


In [93]:
mlflow_client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)
mlflow_client.get_experiment(experiment_id=1)

<Experiment: artifact_location='./mlruns/1', creation_time=1680700375622, experiment_id='1', last_update_time=1680700375622, lifecycle_stage='active', name='demo-experiment-1', tags={}>

In [106]:
experiment = mlflow_client.get_experiment_by_name(TRAINING_EXPERIMENT_NAME)
experiment

<Experiment: artifact_location='./mlruns/1', creation_time=1680700375622, experiment_id='1', last_update_time=1680700375622, lifecycle_stage='active', name='demo-experiment-1', tags={}>

In [107]:
runs = mlflow_client.search_runs(
    experiment_ids=experiment.experiment_id,
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"])

In [171]:
runs[4].data.params

{'alpha': '0.9',
 'ccp_alpha': '0.0',
 'criterion': 'friedman_mse',
 'init': 'None',
 'learning_rate': '0.1',
 'loss': 'squared_error',
 'max_depth': '3',
 'max_features': 'None',
 'max_leaf_nodes': 'None',
 'min_impurity_decrease': '0.0',
 'min_samples_leaf': '1',
 'min_samples_split': '2',
 'min_weight_fraction_leaf': '0.0',
 'n_estimators': '100',
 'n_iter_no_change': 'None',
 'random_state': 'None',
 'subsample': '1.0',
 'tol': '0.0001',
 'validation_fraction': '0.1',
 'verbose': '0',
 'warm_start': 'False'}

In [177]:
list_params = []
for run in runs:
    print(f"estimator model : {run.data.tags['estimator_class']}, estimator_name : {run.data.tags['estimator_name']}, paramters:{run.data.params['max_features']}")
    params = fix_param_datatypes(run.data.params)
    list_params.append(params)
    

estimator model : sklearn.ensemble._forest.RandomForestRegressor, estimator_name : RandomForestRegressor, paramters:1.0
estimator model : sklearn.ensemble._forest.RandomForestRegressor, estimator_name : RandomForestRegressor, paramters:1.0
estimator model : sklearn.ensemble._forest.RandomForestRegressor, estimator_name : RandomForestRegressor, paramters:1.0
estimator model : sklearn.ensemble._forest.ExtraTreesRegressor, estimator_name : ExtraTreesRegressor, paramters:1.0
estimator model : sklearn.ensemble._gb.GradientBoostingRegressor, estimator_name : GradientBoostingRegressor, paramters:None


In [178]:
list_params[4]

{'alpha': 0.9,
 'ccp_alpha': 0.0,
 'criterion': 'friedman_mse',
 'init': 'None',
 'learning_rate': 0.1,
 'loss': 'squared_error',
 'max_depth': 3.0,
 'max_features': 'None',
 'max_leaf_nodes': 'None',
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1.0,
 'min_samples_split': 2.0,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100.0,
 'n_iter_no_change': 'None',
 'random_state': 'None',
 'subsample': 1.0,
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': 0.0,
 'warm_start': 'False'}

In [141]:
param_dict = list_params[0]
param_dict.values()

dict_values(['True', '0.0', 'squared_error', 'None', '1.0', 'None', 'None', '0.0', '1', '2', '0.0', '100', 'None', 'False', 'None', '0', 'False'])

In [148]:
param_dict = list_params[0]
param_dict.values()
correct_param_dict={}
for k, v in param_dict.items():
    print(f"{k} : {v}")
    if v != 'None' and v != 'True' and v != 'False' and v != 'squared_error':
        correct_param_dict[k] = float(v)
    else:
        correct_param_dict[k] = str(v)

bootstrap : True
ccp_alpha : 0.0
criterion : squared_error
max_depth : None
max_features : 1.0
max_leaf_nodes : None
max_samples : None
min_impurity_decrease : 0.0
min_samples_leaf : 1
min_samples_split : 2
min_weight_fraction_leaf : 0.0
n_estimators : 100
n_jobs : None
oob_score : False
random_state : None
verbose : 0
warm_start : False


In [151]:
type(correct_param_dict['warm_start'])

str

In [46]:
pool_of_models = {
       "RandomForestRegressor":RandomForestRegressor,
       "GradientBoostingRegressor":GradientBoostingRegressor,
       "AdaBoostRegressor":AdaBoostRegressor,
       "ExtraTreesRegressor":ExtraTreesRegressor,
       "SVR":SVR,
       "XGBRegressor":XGBRegressor,
       "DecisionTreeRegressor":DecisionTreeRegressor,
       "KNeighborsRegressor":KNeighborsRegressor,
       "LinearRegression":LinearRegression,
       "Ridge":Ridge,
       "Lasso":Lasso
   }
pool_of_models["Lasso"]

sklearn.linear_model._coordinate_descent.Lasso

In [30]:
hey={}
hey= pool_of_models
hey

{'RandomForestRegressor': sklearn.ensemble._forest.RandomForestRegressor,
 'GradientBoostingRegressor': sklearn.ensemble._gb.GradientBoostingRegressor,
 'AdaBoostRegressor': sklearn.ensemble._weight_boosting.AdaBoostRegressor,
 'ExtraTreesRegressor': sklearn.ensemble._forest.ExtraTreesRegressor,
 'SVR': sklearn.svm._classes.SVR,
 'XGBRegressor': xgboost.sklearn.XGBRegressor,
 'DecisionTreeRegressor': sklearn.tree._classes.DecisionTreeRegressor,
 'KNeighborsRegressor': sklearn.neighbors._regression.KNeighborsRegressor,
 'LinearRegression': sklearn.linear_model._base.LinearRegression,
 'Ridge': sklearn.linear_model._ridge.Ridge,
 'Lasso': sklearn.linear_model._coordinate_descent.Lasso}

In [31]:
current_experiment = mlflow_client.get_experiment_by_name(EXPERIMENT_NAME)
current_experiment

<Experiment: artifact_location='./mlruns/2', creation_time=1680706923403, experiment_id='2', last_update_time=1680706923403, lifecycle_stage='active', name='hyper-optimization-experiment-1', tags={}>

### Model Tuning

In [189]:
def fine_tune_model(TRAINING_EXPERIMENT_NAME: str, X_train:pd.DataFrame, y_train:pd.Series, X_val:pd.DataFrame, y_val:pd.Series)->Output(
    model_name=str
):
    '''TRAINING_EXPERIMENT_NAME: the previous experiment for training
    log_top_models: number of the top runs to get
     X_train, X_test,y_train, y_test : for training and testing the models with new parameters
    '''
    
    # get the  mlflow client that will enable us to communicate with the backend tracking uri to get the previous runs
    mlflow_client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)
    # get the experimeent of training  by name
    experiment = mlflow_client.get_experiment_by_name(TRAINING_EXPERIMENT_NAME)
    # get teh runs of the previous experiment
    runs = mlflow_client.search_runs(
        experiment_ids=experiment.experiment_id,
        run_view_type=ViewType.ACTIVE_ONLY,
        max_results=5,
        order_by=["metrics.rmse ASC"]
    )
    # from the best top 5 or 10 runs , we get the parameters for that run and pass it to the optimze functon
    for run in runs:
        optimize_model(X_train,y_train,X_val,y_val,run.data.params,run.data.tags)

    # select the model with the lowest test RMSE
    # get the best performing model from the optimzed models, here we first get the name of the current hyperparamter tuning experiment
    current_experiment = mlflow_client.get_experiment_by_name(EXPERIMENT_NAME)
    # get the best runs since they hav been optimized or tuned, from the list get the top run
    # best_run = mlflow_client.search_runs(
    #     experiment_ids="2",
    #     run_view_type=ViewType.ACTIVE_ONLY,
    #     max_results=1,
    #     order_by=["metrics.rmse ASC"]
    # )[3]
    # model_name="laptop-predicition-modelV1"
    # # register the best model(top of the list model)
    # best_run_info = f"best_run_info : {best_run.info}"
    # # mlflow.register_model(
    # #     model_uri=model_uri, 
    # #     name=model_name
    # # )
    # return model_name, best_run_info

In [186]:
# assign as value pool space for different parameters where the models will take different parameters from
SPACE = {
    'max_depth': scope.int(hp.quniform('max_depth', 1, 20, 1)),
    'n_estimators': scope.int(hp.normal('n_estimators', 10, 50, 1)),
    'min_samples_split': scope.int(hp.quniform('min_samples_split', 2, 10, 1)),
    'min_samples_leaf': scope.int(hp.quniform('min_samples_leaf', 1, 4, 1)),
    'random_state': 42
}

In [190]:
fine_tune_model(TRAINING_EXPERIMENT_NAME,X_train,y_train,X_test,y_test)

2023/04/07 18:58:12 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Anaconda\envs\machine-learning-env\lib\site-packages\mlflow\utils\autologging_utils\safety.py:533: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel()."
2023/04/07 19:00:06 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Anaconda\envs\machine-learning-env\lib\site-packages\mlflow\utils\autologging_utils\safety.py:533: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel()."
2023/04/07 19:00:42 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Anaconda\envs\machine-learning-env\lib\site-packages\mlflow\utils\autologging_utils\safety.py:533: DataConversionWarning: A column-vector y was passed when a 1d array was e

### Model registry MLFlow

##### Interacting with Tracking Server

In [50]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///demo.db"

In [51]:
# interacting with the mlflow beckend tracker\ tracking server
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

client.get_experiment(experiment_id=2)

<Experiment: artifact_location='./mlruns/2', creation_time=1680706923403, experiment_id='2', last_update_time=1680706923403, lifecycle_stage='active', name='hyper-optimization-experiment-1', tags={}>

In [52]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='2',
    filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=2,
    order_by=["metrics.rmse ASC"]
)[1]

In [53]:
runs.info.run_id

'c10ad4177dad4049b3076f01582d130e'

In [8]:
list_of_runs = []
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")
    list_of_runs.append(run.info.run_id)

run id: 128731dce22e4a06bed20def94cf81db, rmse: 0.2046
run id: c10ad4177dad4049b3076f01582d130e, rmse: 0.2061
run id: 987e3045903948a7851fe036a4b672be, rmse: 0.2075
run id: 3f179af39a0d4edb955d567a37826322, rmse: 0.2076
run id: cdeb83a5a7874fa28e332ed2c2ac4f84, rmse: 0.2077


In [56]:
#best_run = list_of_runs[0]
best_run = runs.info.run_id
best_run

'c10ad4177dad4049b3076f01582d130e'

##### Interacting with Model Registry

In [57]:
import mlflow
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [65]:
#run_id = "b8904012c84343b5bf8ee72aa8f0f402"
model_uri = f"runs:/{best_run}/model"
mlflow.register_model(model_uri=model_uri, name="laptop-price-demoV2")

Successfully registered model 'laptop-price-demoV2'.
2023/04/10 13:02:41 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: laptop-price-demoV2, version 1
Created version '1' of model 'laptop-price-demoV2'.


<ModelVersion: creation_timestamp=1681124561823, current_stage='None', description=None, last_updated_timestamp=1681124561823, name='laptop-price-demoV2', run_id='c10ad4177dad4049b3076f01582d130e', run_link=None, source='./mlruns/2/c10ad4177dad4049b3076f01582d130e/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [67]:
model_name = "laptop-price-demoV2"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")
    model_ver = version.version
print(model_ver)

version: 1, stage: None
1


In [19]:
model_version = 1
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)


<ModelVersion: creation_timestamp=1681116727851, current_stage='Staging', description=None, last_updated_timestamp=1681117231554, name='laptop-price-demo', run_id='128731dce22e4a06bed20def94cf81db', run_link=None, source='./mlruns/2/128731dce22e4a06bed20def94cf81db/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [20]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: creation_timestamp=1681116727851, current_stage='Staging', description='The model version 1 was transitioned to Staging on 2023-04-10', last_updated_timestamp=1681117281118, name='laptop-price-demo', run_id='128731dce22e4a06bed20def94cf81db', run_link=None, source='./mlruns/2/128731dce22e4a06bed20def94cf81db/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [61]:
MODEL_NAME="laptop-price-demoV1"
HYPERPARAMETER_TUNING_EXPERIMENT = "hyper-optimization-experiment-1"

In [68]:
def register_model(MLFLOW_TRACKING_URI, MODEL_NAME):
    """ this function is responsible for taking the model in tuning experiment to model registry
    MLFLOW_TRACKING_URI : is the sqlite database used by the tracking server to track the experiments
    """
    # interacting with the mlflow beckend tracker\ tracking server
    client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)
    # client.get_experiment(experiment_id=2)

    best_run = client.search_runs(
    experiment_ids='2',
    filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=2,
    order_by=["metrics.rmse ASC"])[1]

    # interacting with model registry
    mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
    
    model_uri = f"runs:/{best_run.info.run_id}/model"
    mlflow.register_model(model_uri=model_uri, name=MODEL_NAME)
    
    latest_versions = client.get_latest_versions(name=MODEL_NAME)
    for version in latest_versions:
        model_version = version.version
        
    # registering the fresh new model into staging
    new_stage = "Staging"
    client.transition_model_version_stage(
        name=MODEL_NAME,
        version=model_version,
        stage=new_stage,
        archive_existing_versions=False
    )
    
    # update the mlflow client
    date = datetime.today().date()
    client.update_model_version(
        name=MODEL_NAME,
        version=model_version,
        description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
    )

In [69]:
register_model(MLFLOW_TRACKING_URI,MODEL_NAME)

Registered model 'laptop-price-demoV1' already exists. Creating a new version of this model...
2023/04/10 13:04:28 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: laptop-price-demoV1, version 2
Created version '2' of model 'laptop-price-demoV1'.


### Comparing model versions and selecting the best for production

In [ ]:
def get_evaluation_data():
    """ it gets the evalutiion data that has not been touched or tempered from the dataset repo and
    uses it to evaluate the staging model against the production model(IF the production model is present), if not the 
    staging model automatically moves to  productionn label and and gets deployed.
    NB: In case where the two models are compared(Prod vs Staging), the better model is deployed and the currently in prod
    moddel gets removed to archivedd models.
    """